<a href="https://colab.research.google.com/github/teploe-odealko/meteoinfo_parsing/blob/main/%D0%9A%D0%BE%D0%B2%D0%B8%D0%B4_%D1%80%D0%BE%D1%81%D1%81%D0%B8%D0%B8%D1%8F_xlsx_%2B_%D0%BF%D1%80%D0%B5%D0%B7%D0%B5%D0%BD%D1%82%D0%B0%D1%86%D0%B8%D1%8F_%2B_%D0%BF%D0%BE%D0%B3%D0%BE%D0%B4%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Настройка

In [1]:
hours = 12

In [2]:
hours_button_map = {12: 'id_l_1', 24: 'id_l_2', 36: 'id_l_3'}
try:
    hours_button_id = hours_button_map[hours]
except:
    raise KeyError('Неправильное значение переменной "hours"')

сводка ковида по России (xlsx файл)

In [3]:
from google.colab import files
covid_russia_filename = files.upload()
covid_russia_filename = next(iter(covid_russia_filename))

Saving russia_regions.xlsx to russia_regions.xlsx


Шаблон meteoinfo (tr_template.xlsx)

In [4]:
from google.colab import files
meteoinfo_filename = files.upload()
meteoinfo_filename = next(iter(meteoinfo_filename))

Saving rt_template.xlsx to rt_template.xlsx


презентация

In [5]:
from google.colab import files
pptx_filename = files.upload()
pptx_filename = next(iter(pptx_filename))

Saving 13.pptx to 13.pptx


#Скрипт

##Россия excel

In [ ]:
!pip install selenium
!pip install python-pptx
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!pip install pip install excel-formulas-calculator
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver


In [7]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [8]:
import openpyxl
from typing import Dict, List, Tuple

from bs4 import BeautifulSoup
import urllib3
from openpyxl.worksheet.worksheet import Worksheet
from nltk.stem.snowball import SnowballStemmer
from datetime import datetime, date, time, timedelta
from openpyxl.formula import Tokenizer
from openpyxl.formula.translate import Translator
from openpyxl.utils.cell import get_column_letter
from copy import copy
from openpyxl.formatting.rule import FormulaRule

import re
from openpyxl.styles import Color, PatternFill, Font, Border
from pptx import Presentation
from efc.interfaces.iopenpyxl import OpenpyxlInterface
import numpy as np
urllib3.disable_warnings()

In [9]:
def get_general_info(info: dict):
    STOP_COVID_URL = 'https://стопкоронавирус.рф/information/'
    wd.get(STOP_COVID_URL)
    raw_rows = wd.find_elements_by_xpath('//h3[@class="cv-stats-virus__item-value"]')
    rows = [row.text for row in raw_rows]
    statistics = [int(''.join(row.strip('\n ').split())) for row in rows]
    useful_statistics = [statistics[2], statistics[4], statistics[0]]
    info.update({'general': useful_statistics})

In [10]:
URL = 'https://yastat.net/s3/milab/2020/covid19-stat/data/v10/deep_data.json'
REGION_COLUMN = 2
REGION_ROWS_RANGE = (2, 87)
WEEKLY_REPORT_DAY = 7
DAYS_OF_THE_WEEK_MAP = {
    1: 'ПН',
    2: 'ВТ',
    3: 'СР',
    4: 'ЧТ',
    5: 'ПТ',
    6: 'СБ',
    7: 'ВС',
}

def normolize_string(string: str):
    string = string.strip().lower().translate({ord('.'): None})
    string = ' '.join(string.split('-'))

    # remove stop words
    stop_words = ['г', 'обл', 'область', 'республика', 'автономная', 'ао']
    try:
        splitted_string = string.split()
    except IndexError:
        return ''
    stemmer = SnowballStemmer("russian")
    string = ' '.join(stemmer.stem(w) for w in splitted_string if w not in stop_words)
    return string


def fill_сases_sheet(ws: Worksheet, info: dict):
    TOTAL_CASES_FORMULA_ROW = 88

    last_available_date = datetime.strptime(info['dates'][-1], '%Y-%m-%d')
    last_column = ws.max_column
    last_table_date = ws.cell(row=1, column=last_column).value
    new_days_amount = (last_available_date - last_table_date).days
    for i in range(1, new_days_amount + 1):
        ws.cell(row=1, column=(last_column + i)).value = (last_table_date + timedelta(days=i))
        ws.cell(row=1, column=(last_column + i)).number_format = 'DD.MM.YYYY'

        for row in range(*REGION_ROWS_RANGE):
            name = ws.cell(row=row, column=REGION_COLUMN).value
            key = normolize_string(name)
            if key in info['data']:
                ws.cell(row=row, column=(last_column + i)).value = \
                    info['data'][key]['cases'][(i - 1) - new_days_amount][0]
            else:
                raise KeyError(key)

        formula = ws.cell(row=TOTAL_CASES_FORMULA_ROW, column=ws.max_column - 1).value
        last_cell_id = get_column_letter(ws.max_column - 1) + str(TOTAL_CASES_FORMULA_ROW)
        next_cell_id = get_column_letter(ws.max_column) + str(TOTAL_CASES_FORMULA_ROW)
        ws[next_cell_id] = Translator(formula, origin=last_cell_id).translate_formula(next_cell_id)

    info.update({'new_days_amount': new_days_amount})

import requests
def get_regions_info() -> dict:
    final_region_data = {}

    response: dict = requests.get(URL, verify=False).json()
    regions = response['russia_stat_struct']['data']
    dates = response['russia_stat_struct']['dates']
    for region in regions:
        full_name = regions[region]['info']['name']
        short_name = regions[region]['info']['short_name']
        regions[region]['info'].pop('name', None)
        regions[region]['info'].pop('short_name', None)
        final_region_data.update(
            dict.fromkeys([normolize_string(full_name), normolize_string(short_name)], regions[region]))
    return {'data': final_region_data,
            'dates': dates,
            'last_date': datetime.strptime(dates[-1], '%Y-%m-%d')}


def continue_formula_right(ws: Worksheet, rows: list, column: int):
    for row in rows:
        formula = ws.cell(row=row, column=column - 1).value
        last_cell_id = get_column_letter(column - 1) + str(row)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id] = Translator(formula, origin=last_cell_id).translate_formula(next_cell_id)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def  continue_date_right(ws: Worksheet, rows: list):
    for row in rows:
        try:
            column = [c.value for c in ws[row]][3:].index(None) + 4
        except ValueError:
            column = ws.max_column + 1
        last_table_date = ws.cell(row=row, column=column - 1).value
        ws.cell(row=row, column=column).value = (last_table_date) + timedelta(days=1)
        ws.cell(row=row, column=column).number_format = 'DD.MM.YYYY'

        last_cell_id = get_column_letter(column - 1) + str(row)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def continue_number_right(ws: Worksheet, rows: list):
    for row in rows:
        try:
            column = [c.value for c in ws[row]].index(None) + 1
        except ValueError:
            column = ws.max_column + 1
        last_num = ws.cell(row=row, column=column - 1).value
        ws.cell(row=row, column=column).value = last_num + 1

        last_cell_id = get_column_letter(column - 1) + str(row)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def gain_sheet(ws: Worksheet, info: dict):
    NEW_COLUMN_NUM_GAIN = [c.value for c in ws[1]].index(None) + 1

    for i in range(info['new_days_amount']):
        ws.insert_cols(NEW_COLUMN_NUM_GAIN+i)
        continue_date_right(ws, [1])
        continue_formula_right(ws, list(range(2, 96)), NEW_COLUMN_NUM_GAIN + i)

    info['last_col_in_gain_sheet'] = [c.value for c in ws[1]].index(None)


def daily_region_gain_sheet(ws: Worksheet, info: dict):
    NEW_COLUMN_NUM_DAILY_GAIN = ws.max_column + 1

    for i in range(info['new_days_amount']):
        continue_date_right(ws, [1, 20, 35])
        continue_formula_right(ws,
                               list(range(2, 19)) + list(range(21, 34)) + list(range(36, 49)),
                               NEW_COLUMN_NUM_DAILY_GAIN + i)


def weekly_gain_sheet(ws: Worksheet, info: dict):
    new_col = [c.value for c in ws[1]].index(None) + 1
    ws.insert_cols(new_col)
    continue_number_right(ws, [1])
    continue_formula_right(ws, list(range(2, 94)), new_col)
    for row in range(2, 87):
        # left part
        raw_formula = ws.cell(row=row, column=new_col).value
        tokenized_formula = Tokenizer(raw_formula)
        last_col_in_gain_sheet = info['last_col_in_gain_sheet']

        tokenized_formula.items[1].value = re.sub(r'(?<=:\$)\D*',
                                                  get_column_letter(last_col_in_gain_sheet),
                                                  tokenized_formula.items[1].value)
        tokenized_formula.items[3].value = re.sub(r'(?<=:\$).*(?=\$)',
                                                  get_column_letter(last_col_in_gain_sheet),
                                                  tokenized_formula.items[3].value)
        new_formula = ''.join(token.value for token in tokenized_formula.items)
        ws.cell(row=row, column=new_col).value = fr'={new_formula}'

        # right part
        ws.cell(row=row, column=new_col+2).value = fr'=MAX($C{row}:${get_column_letter(new_col)}{row})'
        ws.cell(row=row, column=new_col+3).value = fr'={get_column_letter(new_col+2)}{row}-{get_column_letter(new_col)}{row}'
        ws.cell(row=row, column=new_col+4).value = fr'={get_column_letter(new_col)}{row}-{get_column_letter(new_col-1)}{row}'
        ws.cell(row=row, column=new_col+6).value = fr'={get_column_letter(new_col)}{row}/{get_column_letter(new_col-1)}{row}-100%'
        ws.cell(row=row, column=new_col+7).value = fr'={get_column_letter(new_col-1)}{row}/{get_column_letter(new_col-2)}{row}-100%'


    ws.cell(row=87, column=new_col+4).value = fr'={get_column_letter(new_col)}{87}-{get_column_letter(new_col-1)}{87}'


def tpr_weekly_gain_sheet(ws: Worksheet, info: dict):
    new_col = [c.value for c in ws[1]].index(None) + 1
    ws.insert_cols(new_col)
    continue_number_right(ws, [1])
    continue_formula_right(ws, list(range(2, 88)), new_col)


# {}

# def cases_sheet(wb):
#
#
    # continue_number_right(ws_weekly_gain, )


def continue_formula_n_down(ws: Worksheet, columns: list, row: int, n: int):
    for column in columns:

        formula = ws.cell(row=(row - n), column=column).value
        last_cell_id = get_column_letter(column) + str(row - n)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id] = Translator(formula, origin=last_cell_id).translate_formula(next_cell_id)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def continue_formula_down(ws: Worksheet, columns: list, row: int):
    continue_formula_n_down(ws, columns, row, 1)


# def get_first_clear_column_in_row(row):
#     return max((c.row for c in column if c.value is not None)) + 1


def get_first_clear_row_in_column(column):
    return max((c.row for c in column if c.value is not None)) + 1

def base_rf_sheet(ws: Worksheet, info: dict):
    NEW_ROW_NUM = get_first_clear_row_in_column(ws['B'])

    ws.cell(row=NEW_ROW_NUM, column=1).value = info['last_date']
    ws.cell(row=NEW_ROW_NUM, column=1).number_format = 'DD.MM'
    for i, stat in enumerate(info['general']):
        ws.cell(row=NEW_ROW_NUM, column=2 + i).value = info['general'][i]
        ws.cell(row=NEW_ROW_NUM, column=2 + i)._style = copy(ws.cell(row=NEW_ROW_NUM - 1, column=2 + i)._style)
    continue_formula_down(ws,
                          list(range(5, 20)) + [26] + list(range(28, 36)),
                          NEW_ROW_NUM)

    if info['last_date'].weekday() + 1 == WEEKLY_REPORT_DAY:  # if it's sunday
        continue_formula_n_down(ws, list(range(20, 25))+[27], NEW_ROW_NUM, 7)
        continue_formula_n_down(ws, list(range(20, 23)), NEW_ROW_NUM + 1, 7)
    I_column = [c.value for c in ws['I'][10:]]
    info['day_infections_cell'] = f'I{I_column.index(None)+10}'


def date_week_sheet(ws: Worksheet, info: dict):
    NEW_ROW_NUM = get_first_clear_row_in_column(ws['A'])
    ws.cell(row=NEW_ROW_NUM, column=1).value = ws.cell(row=NEW_ROW_NUM - 1, column=1).value + 1
    continue_formula_down(ws, list(range(2, 4)), NEW_ROW_NUM)




def parse_info():
    info = get_regions_info()
    # {dates: [list of available dates], data: {'*REGION_NAME*': {*REGION_INFO*}}}
    get_general_info(info)
    return info


def weekly_region_gain_sheet(ws: Worksheet, info: dict):
    new_column = ws.max_column + 1

    continue_number_right(ws, [1, 20])
    continue_formula_right(ws,
                           list(range(2, 17)) + list(range(21, 34)),
                           new_column)

def gain_7day_sheet(ws: Worksheet, info: dict):
    new_column = [c.value for c in ws[1]].index(None) + 1

    for i in range(info['new_days_amount']):
        ws.insert_cols(new_column+i)
        continue_date_right(ws, [1])
        continue_formula_right(ws, list(range(2, 94)), new_column + i)

    for row in range(2, 87):
        ws.cell(row=row, column=new_column+i+3).value = fr'={get_column_letter(new_column+i)}{row}'
        ws.cell(row=row, column=new_column+i+2).value = fr'=RANK({get_column_letter(new_column+i+3)}{row},${get_column_letter(new_column+i+3)}$2:${get_column_letter(new_column+i+3)}$86,0)'
        ws.cell(row=row, column=new_column+i+4).value = fr'={get_column_letter(new_column+i)}{row}-{get_column_letter(new_column+i-7)}{row}'
        ws.cell(row=row, column=new_column+i+6).value = fr'={get_column_letter(new_column+i)}{row}/{get_column_letter(new_column+i-7)}{row}-1'
    ws.cell(row=row, column=new_column+i+3).value = fr'={get_column_letter(new_column + i)}{row}'

    for row in range(92, 97):
        ws.cell(row=row, column=new_column+i+4).value = fr'=VLOOKUP({get_column_letter(new_column+i+2)}{row},${get_column_letter(new_column+i+2)}$2:${get_column_letter(new_column+i+3)}$86,2,0)'
        ws.cell(row=row, column=new_column+i+5).value = fr'={get_column_letter(new_column+i+4)}{row}/${get_column_letter(new_column+i+4)}$97'
        ws.cell(row=row, column=new_column+i+6).value = fr'==VLOOKUP({get_column_letter(new_column+i+2)}{row},${get_column_letter(new_column+i+2)}$2:${get_column_letter(new_column+i+6)}$86,4,0)'
    ws.cell(row=97, column=new_column+i+4).value = fr'={get_column_letter(new_column+i+3)}87'
    ws.cell(row=97, column=new_column+i+5).value = fr'={get_column_letter(new_column+i+4)}{97}/${get_column_letter(new_column+i+4)}$97'

def rt_sheet(ws: Worksheet, info: dict):
    new_column = [c.value for c in ws[2]][3:].index(None) + 4
    for i in range(info['new_days_amount']):
        ws.insert_cols(new_column+i)
        continue_date_right(ws, [2])
        continue_formula_right(ws, list(range(3, 89)), new_column + i)

    ws.cell(row=1, column=(new_column+i+5)).value = info['last_date']
    c = get_column_letter(new_column+i+5)
    r = '1'
    cell1 = c+r
    ws.cell(row=1, column=(new_column+i+6)).value = fr'=MATCH({cell1},$A$2:{get_column_letter(new_column+i)}2)-3'

    for row in range(3, 88):
        col = get_column_letter(new_column+i)
        ws.cell(row=row, column=(new_column+i+2)).value = fr'=RANK({col}{row},${col}$3:${col}$87,0)'

        ws.cell(row=row, column=(new_column+i+4)).value = fr'=IF(INDEX($D$3:${col}$87,${get_column_letter(new_column+i+8)}{row},${get_column_letter(new_column+i+6)}$1)<1,1,0)'
        ws.cell(row=row, column=(new_column+i+5)).value = fr'=IF(INDEX($D$3:${col}$87,${get_column_letter(new_column+i+8)}{row},${get_column_letter(new_column+i+6)}$1)>1,1,0)'
        ws.cell(row=row, column=(new_column+i+6)).value = fr'=IF(INDEX($D$3:${col}$87,${get_column_letter(new_column+i+8)}{row},${get_column_letter(new_column+i+6)}$1)=1,1,0)'

    ws.cell(row=88, column=(new_column+i+4)).value = fr'=SUM({get_column_letter(new_column+i+4)}3:{get_column_letter(new_column+i+4)}87)'
    ws.cell(row=88, column=(new_column+i+5)).value = fr'=SUM({get_column_letter(new_column+i+5)}3:{get_column_letter(new_column+i+5)}87)'
    ws.cell(row=88, column=(new_column+i+6)).value = fr'=SUM({get_column_letter(new_column+i+6)}3:{get_column_letter(new_column+i+6)}87)'
    ws.cell(row=89, column=(new_column+i+6)).value = fr'=85-SUM({get_column_letter(new_column+i+4)}88:{get_column_letter(new_column+i+6)}88)'
    info['rt_bigger1_cell'] = f'{get_column_letter(new_column+i+5)}88'


def delta_day_sheet(ws: Worksheet, info: dict):
    new_column = ws.max_column + 1
    bigger_zero_fill = PatternFill(start_color='F1CCB1', end_color='F1CCB1', fill_type='solid')
    lower_zero_fill = PatternFill(start_color='E4EFDC', end_color='E4EFDC', fill_type='solid')

    for i in range(info['new_days_amount']):
        continue_date_right(ws, [1])
        ws.cell(row=2, column=new_column + i).value = DAYS_OF_THE_WEEK_MAP.get(
            ws.cell(row=1, column=new_column + i).value.weekday() + 1
        )
        last_cell_id = get_column_letter(new_column + i - 1) + str(2)
        next_cell_id = get_column_letter(new_column + i) + str(2)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)

        continue_formula_right(ws, list(range(3, 98)), new_column + i)
        cell_ids_formatting = '{}3:{}87'.format(get_column_letter(new_column + i), get_column_letter(new_column + i))
        ws.conditional_formatting.add(cell_ids_formatting, FormulaRule(formula=[f'{next_cell_id}>0'], fill=bigger_zero_fill))
        ws.conditional_formatting.add(cell_ids_formatting, FormulaRule(formula=[f'{next_cell_id}<0'], fill=lower_zero_fill))


def msc_sp_sheet(ws: Worksheet, info: dict):
    new_column = ws.max_column + 1
    # bigger_zero_fill = PatternFill(start_color='F1CCB1', end_color='F1CCB1', fill_type='solid')
    # lower_zero_fill = PatternFill(start_color='E4EFDC', end_color='E4EFDC', fill_type='solid')

    for i in range(info['new_days_amount']):
        continue_date_right(ws, [1])
        ws.cell(row=2, column=new_column + i).value = DAYS_OF_THE_WEEK_MAP.get(
            ws.cell(row=1, column=new_column + i).value.weekday() + 1
        )
        last_cell_id = get_column_letter(new_column + i - 1) + str(2)
        next_cell_id = get_column_letter(new_column + i) + str(2)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)
        continue_formula_right(ws, list(range(3, 98)), new_column + i)


In [11]:
def covid_russia_xlsx(filename):
    wb = openpyxl.load_workbook(filename)
    
    info = parse_info()
    info['covid_russia_wb'] = wb
    fill_сases_sheet(wb['Случаев'], info)
    gain_sheet(wb['Прирост'], info)
    msc_sp_sheet(wb['МСК и СП'], info)
    rt_sheet(wb['Rt'], info)
    gain_7day_sheet(wb['прирост 7дн'], info)
    delta_day_sheet(wb['дельта за сутки'], info)
    daily_region_gain_sheet(wb['По рег прис (сут)'], info)

    if info['last_date'].weekday()+1 == WEEKLY_REPORT_DAY:
        weekly_gain_sheet(wb['Прирост нед'], info)
        tpr_weekly_gain_sheet(wb['Тпр нед прироста'], info)
        weekly_region_gain_sheet(wb['По рег прис (нед)'], info)
        date_week_sheet(wb['Дата-неделя'], info)

    base_rf_sheet(wb['База РФ'], info)


    out_filename = filename.split('.')[0] + '_out.' + filename.split('.')[1]
    wb.save(out_filename)
    info['files_to_download'] = [out_filename]
    return info

##Презентация

In [12]:

def fill_pptx(filename:str, info: dict):
    
    prs = Presentation(filename)
    interface = OpenpyxlInterface(wb=info['covid_russia_wb'], use_cache=True)
    day_infections = interface.calc_cell(info['day_infections_cell'], 'База РФ')
    rt_bigger_1_num = interface.calc_cell(info['rt_bigger1_cell'], 'Rt')
    s1 = "субъекте" if (rt_bigger_1_num%10 == 1) and rt_bigger_1_num != 11 else "субъектах"

    gain = []
    regions = []
    for i in range(2, 87):
        region = interface.calc_cell(f'B{i}', 'Прирост')
        if region not in ['Санкт-Петербург', 'Московская область', 'Москва']:
            gain.append(interface.calc_cell(f'{get_column_letter(info["last_col_in_gain_sheet"])}{i}', 'Прирост'))
            regions.append(region)
    top5_idx = np.array(gain).argsort()[-5:][::-1]
    top5_region = np.array(regions)[top5_idx]
    top5_gain = np.array(gain)[top5_idx]
    s2 = ', '.join([f'{region} ({gain})' for region, gain in zip(top5_region, top5_gain)])
    template = f"{day_infections} новых случаев в России. Ухудшение – в {rt_bigger_1_num} {s1} РФ. Наибольший прирост новых случаев: {s2}."
    # print(template)
    info['covid_yandex_info'] = template
    prs.slides[1].shapes[11].text_frame.paragraphs[1].runs[0].text = template

    out_filename = filename.split('.')[0] + '_out.' + filename.split('.')[1]
    prs.save(out_filename)

    info['files_to_download'].append(out_filename)

##Meteoinfo погодные явления

In [13]:
!pip install requests-html

     |████████████████████████████████| 87 kB 4.0 MB/s 
     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 76 kB 4.2 MB/s 
     |████████████████████████████████| 138 kB 37.1 MB/s 
     |████████████████████████████████| 127 kB 37.0 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13503 sha256=93d0acd9274aec181e6bdd9cb19b6b91931a8ae59144d5fd6ce1f4d4ce7f0692
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=8a093e59d3ed4b092bbb5336ef584d9925a9381ebe62d87355465b9cc52ac182
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built fake-useragent parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled ur

In [14]:
#!/usr/bin/env python3
import collections
from typing import Dict, List
from collections import defaultdict
from openpyxl.worksheet.worksheet import Worksheet
import sys
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import urllib3
import requests
import openpyxl

import re
import warnings
import requests
from openpyxl import load_workbook
from datetime import datetime

warnings.filterwarnings('ignore')
urllib3.disable_warnings()


URLS = [
    'https://meteoinfo.ru/hdmapsmeteoalarmszfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmcfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmprfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmyugfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmyskfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmural',
    'https://meteoinfo.ru/hdmapsmeteoalarmsyb',
    'https://meteoinfo.ru/hdmapsmeteoalarmdv',
]

HAZARDS_URL = 'https://meteoinfo.ru/hazardsbull'

SIZE = 4

SHEET_NAME = 'Билет по регионам'

REGION_COLUMN = 2

FORECAST_COLUMNS = (6, 7, 8, 9)
FACTOR_COLUMNS = (10, 11, 12, 13)

FORCASTS = (
    'Угроза повреждения ЛЭП и линий связи, обрушения слабо укрепленных, широкоформатных, ветхих и рекламных конструкций',
    'Угроза выхода из строя систем ЖКХ, городских коммуникаций',
    'Угроза затопления населенных пунктов, дорог, мостов',
    'Угроза сбоев в работе всех видов транспорта, увеличение количества ДТП',
    'Угроза жизни и здоровью населения',
    'Увеличение количества травм пешеходов',
    'Угроза повреждения населенных пунктов, дорог, мостов'
)

# INTERCHANGEABLE_FACTORS = [
#     {'Ветер', 'Дождь', 'Снег/Обледенение', 'Гроза'},
#     {'Заморозки', 'Очень низкая температура'},
#     {'Туман', 'Голеледно - изморозевые отложения'},
#     {'Пожарная опасность'},
#     {'Паводок'},
#     {'Очень высокая температура'}
# ]

FORCAST_MAP = {
    'очень высокая температура': set((FORCASTS[4],)),
    'очень низкая температура': set((FORCASTS[1], FORCASTS[4])),
    'заморозки': set((FORCASTS[1], FORCASTS[4])),
    'ветер': set((FORCASTS[0], FORCASTS[1], FORCASTS[4])),
    'дождь': set((FORCASTS[0], FORCASTS[1])),
    'гроза': set((FORCASTS[0], FORCASTS[1])),
    'град': set((FORCASTS[0], FORCASTS[1])),
    'паводок': set((FORCASTS[2], FORCASTS[4])),
    'наводнение': set((FORCASTS[2], FORCASTS[4])),
    'снег/Обледенение': set((FORCASTS[0], FORCASTS[1], FORCASTS[3])),
    'голеледно - изморозевые отложения': set((FORCASTS[3], FORCASTS[5])),
    'лавины': set((FORCASTS[4], FORCASTS[6])),
    'сель': set((FORCASTS[4], FORCASTS[6])),
    'прибрежные события': set((FORCASTS[1], FORCASTS[6], FORCASTS[4])),
    'пыльная (песчаная) буря': set((FORCASTS[0], FORCASTS[1], FORCASTS[3], FORCASTS[4])),
    'туман': set((FORCASTS[3],)),
    'пожарная опасность': set((FORCASTS[6], FORCASTS[4])),
    'прочие опасности': set(),
}

DASHBOARD_LABEL_MAP = {
    'очень высокая температура': 'Критическое повышение температуры воздуха',
    'очень низкая температура': 'Критическое понижение температуры воздуха',
    'заморозки': 'Заморозки',
    'ветер': 'Сильный ветер',
    'дождь': 'Дождь',
    'гроза': 'Гроза',
    'паводок': 'Паводок, подтопление',
    'наводнение': 'Наводнение',
    'снег/Обледенение': 'Снегопад, метель',
    'голеледно - изморозевые отложения': 'Гололедица',
    'лавины': 'Оползни, обвалы, снежные лавины',
    'сель': 'Сель',
    'прибрежные события': 'Прибрежные события',
    'пыльная (песчаная) буря': 'Пыльная (песчаная) буря',
    'туман': 'Туман',
    'пожарная опасность': 'Пожароопасная обстановка, пожары',
    'прочие опасности': 'Прочие опасности',
    'град': 'Град или снежный дождь'
}

FACTORS = FORCAST_MAP.keys()

regions_need_to_join = {'саха',
                        'магаданская',
                        'ямало ненецкий ао',
                        'камчатский край',
                        'амурская',
                        'хабаровский край',
                        'чеч',
                        'северная осетия',
                        'красноярский',
                        'моск'}


def join_region(region: str) -> str:
    if region.startswith('сахалин'):
        return 'сахалин'
    for reg in regions_need_to_join:
        if region.startswith(reg):
            return reg
    return region


def normolize_string(string: str):
    string = string.strip().lower().translate({ord('.'): None})
    string = ' '.join(string.split('-'))

    # remove stop words
    stop_words = ['г', 'обл', 'область', 'республика', 'югра', 'ао']
    try:
        splitted_string = string.split()
    except IndexError:
        return ''
    string = ' '.join(w for w in splitted_string if w not in stop_words)
    return join_region(string)


def remove_interchangeable_factors(factor_desc_list: list):
    factors_set = set()
    prev_len_factors_set = 0

    for factor_desc in factor_desc_list:
        factor = factor_desc[0]
        forecast = FORCAST_MAP.get(factor.lower())
        if not forecast:
            continue
        factors_set.update(forecast)
        if len(factors_set) == prev_len_factors_set:
            factor_desc_list.remove(factor_desc)
            return factor_desc_list
        prev_len_factors_set = len(factors_set)
    return factor_desc_list


def analyze_factors(factors_desc: list):
    final_forecasts = set()

    prev_len = len(factors_desc)
    while len(factors_desc) > SIZE:
        factors_desc = remove_interchangeable_factors(factors_desc)
        if len(factors_desc) == prev_len:  # If nothing has been deleted, means there is no interchangeable factors
            factors_desc = factors_desc[:SIZE]
        prev_len = len(factors_desc)

    for factor, desc in factors_desc:
        forecast: set = FORCAST_MAP.get(factor.lower())
        if forecast is None:
            print(f"\x1b[31mНЕИЗВЕСТНЫЙ ФАКТОР \"{factor}\"\x1b[0m")
            continue
        final_forecasts.update(forecast)

    while len(final_forecasts) > SIZE:
        final_forecasts.pop()

    return final_forecasts, set(factors_desc)


def factor_mapping(factor: str, description: str):
    dashboard_factor = DASHBOARD_LABEL_MAP.get(factor.lower())
    if dashboard_factor == 'Дождь':
        for word in description.split():
            if word.startswith('сильн') or word.startswith('лив'):
                dashboard_factor = 'Сильный дождь, ливневой дождь'
                break
    return dashboard_factor

def fill_excel_regions_weather(ws: Worksheet, regions: dict):
    for row in range(2, ws.max_row + 1):
        name = ws.cell(row=row, column=REGION_COLUMN).value
        if not name:
            continue

        name = normolize_string(name)
        factors_desc = regions.get(name)

        if not factors_desc: # if there is no forecast for the region
            # print(name)
            continue

        forecasts, factors_desc = analyze_factors(factors_desc)
        for column, forecast in zip(FORECAST_COLUMNS, forecasts):
            ws.cell(row=row, column=column).value = forecast

        for column, factor_desc in zip(FACTOR_COLUMNS, factors_desc):
            factor = factor_desc[0]
            desc = factor_desc[1]
            ws.cell(row=row, column=column).value = factor_mapping(factor, desc)


def get_hazards(ws:Worksheet, url: str):
    session = HTMLSession()
    response = session.get(url, verify=False)
    rows = response.html.find('#div_1', first=True).xpath('*/table/tbody/tr')
    row_data = rows[1].xpath('//p', first=True)
    ws['B5'].value = row_data.text
    # print(row_data.text)


def parse_description(factor_list: list, region_factor_desc: str):
    for word in re.split(';|,|\.| ', region_factor_desc):
        # try:
        #     factors = list(zip(*factor_list))[0]
        # except IndexError:
        #     factors = []
        if (word.lower().startswith('дожд') or word.lower().startswith('лив')):
            factor_list.append(('Дождь', region_factor_desc))
        elif word.lower().startswith('гроз'):
            factor_list.append(('Гроза', region_factor_desc))
        elif word.lower().startswith('град'):
            factor_list.append(('Град', region_factor_desc))
        elif (word.lower().startswith('шквал') or word.lower().startswith('вет')):
            factor_list.append(('Ветер', region_factor_desc))


def get_regions(urls: List[str]) -> Dict:
    # factors = defaultdict(list)
    # descriptions = defaultdict(list)
    regions = defaultdict(list)
    for url in urls:
        wd.get(url)
        hours24_button = wd.find_element_by_id(hours_button_id)
        hours24_button.click()
        for row in wd.find_elements_by_xpath('//div[@class="test"]'):
            try:
                row_data_bs = BeautifulSoup(row.get_attribute('data-original-title'),
                                            features='lxml')
            except:
                print('error')
                continue  # if raw_data is None
            region_name = normolize_string(row_data_bs.find(class_='cl_tbl_obl_name_tooltip').text)
            region_factor = row_data_bs.find(class_='cl_tbl_tooltip_text').find('b').text.strip('. ')
            region_factor_desc = '.'.join(row_data_bs.find(class_='cl_tbl_tooltip_text').text.split('.')[1:]).strip()

            if (region_factor == 'КМЯ' or region_factor == 'Прочие опасности'):
                parse_description(regions[region_name], region_factor_desc)
                # print(regions[region_name], region_factor)
                continue

            

            try:
                factors = list(zip(*regions[region_name]))[0]
            except IndexError:
                factors = []

            if region_factor != '':
                regions[region_name].append((region_factor, region_factor_desc))

    return regions


def fill_excel_covid_rt(ws: Worksheet):
    obl_pattern = re.compile(r'\bобл\.', re.IGNORECASE)
    ao_pattern = re.compile(r'(?<!Еврейская)\bао\b', re.IGNORECASE)
    gorod_pattern = re.compile(r'\bг\.', re.IGNORECASE)
    strange_pattern = re.compile(r'[—]')

    url = 'https://yastat.net/s3/milab/2020/covid19-stat/data/v10/deep_data.json?v=1618916098074'
    sheet = 'Билет по регионам'

    json_dict = requests.get(url, verify=False).json()

    region_rt_dict = {strange_pattern.sub('-', x['info']['name'].lower()): x['info'].get('rt') for x in json_dict['russia_stat_struct']['data'].values()}
    region_rt_dict_short_name = {strange_pattern.sub('-', x['info']['short_name'].lower()): x['info'].get('rt') for x in json_dict['russia_stat_struct']['data'].values()}

    for row in range(2, ws.max_row + 1):
        # print(ws.cell(row=row, column=2).value)
        key = ws.cell(row=row, column=2).value.lower()

        key = obl_pattern.sub('область', key).strip()
        key = gorod_pattern.sub('', key).strip()

        value = region_rt_dict_short_name.get(key, region_rt_dict.get(ao_pattern.sub('автономный округ', key).strip()))

        ws.cell(row=row, column=3).value = round(value, 2) if value else value

def clear_table(ws: Worksheet):
    for row in range(2, ws.max_row + 1):
        for column in (FORECAST_COLUMNS + FACTOR_COLUMNS + (3,)):
            ws.cell(row=row, column=column).value = None

In [15]:
def meteoinfo(filename: str, info: dict):
    wb = openpyxl.load_workbook(filename)
    ws = wb[SHEET_NAME]

    clear_table(ws)
    fill_excel_covid_rt(ws=ws)
    regions = get_regions(urls=URLS)
    print(regions)
    # remove duplicate factors
    for region in regions:
        factors = regions[region]
        final_list = []
        for factor in factors:
            try:
                unique_factors = list(zip(*final_list))[0]
            except:
                unique_factors = []
            if factor[0] not in unique_factors:
                final_list.append(factor)
        regions[region] = final_list


    fill_excel_regions_weather(ws=ws, regions=regions)
    get_hazards(wb['Обстановка в банке'], HAZARDS_URL)
    wb['Обстановка в банке']['B7'].value = info['covid_yandex_info']

    today = datetime.today().strftime("%d-%m-%Y")
    splitted_filename = filename.split('.')
    splitted_filename[0] = f'{splitted_filename[0]}_{today}_{hours}'
    output_file_name = '.'.join(splitted_filename)
    wb.save(output_file_name)
    info['files_to_download'].append(output_file_name)
    # return output_file_name

##main

In [18]:
def main(covid_russia_filename: str, pptx_filename: str, meteoinfo_filename: str):
    # Заполнение сводки ковида по россии в xlsx файл
    info = covid_russia_xlsx(covid_russia_filename)
    # Заполнение правого блока второго слайда, связанного с ковидом
    fill_pptx(pptx_filename, info)
    # Заполнение погодных данных с meteoinfo в xlsx файл
    meteoinfo(meteoinfo_filename, info)
    
    # out_file = filename.split('.')
    # out_file[0] = f'{out_file[0]}_saved'
    # out_file = '.'.join(out_file)
    return info


# Результат

In [19]:
out_file = main(covid_russia_filename, pptx_filename, meteoinfo_filename)

defaultdict(<class 'list'>, {'архангельская': [('Пожарная опасность', 'Период предупреждения до 21 ч. 22 июля местами 5 класс пожароопасности (более 3000 градусов по формуле Нестерова)')], 'вологодская': [('Пожарная опасность', 'Период предупреждения до 21 ч. 22 июля местами пожароопасность 4 кл.')], 'калининградская': [], 'ленинградская': [('Ветер', 'Период предупреждения с 12 ч. 22 июля до 21 ч. 22 июля 22.07 во второй половине дня местами усиление ветра до 15-17 м/с.'), ('Пожарная опасность', 'Период предупреждения до 18 ч. 22 июля местами пожароопасность 4 класса'), ('Гроза', 'Период предупреждения с 13 ч. 22 июля до 20 ч. 22 июля 22.07 во второй половине дня местами ожидается гроза.')], 'мурманская': [], 'ненецкий': [('Пожарная опасность', 'Период предупреждения до 21 ч. 22 июля местами высокая пожароопасность 4 класса'), ('Ветер', 'Период предупреждения до 21 ч. 22 июля местами порывы 15-17 м/с, ночью и утром 22 июля на восточном побережье 17-22 м/с')], 'новая земля': [('Ветер', 

In [21]:
for filename in out_file['files_to_download']:
    files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>